# Generate Density Fields

Use pmwd to simulate 10,000 density fields for training/testing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
from jax import jit
from tqdm import tqdm
from pmwd import (
    Configuration,
    Cosmology, SimpleLCDM, Planck18,
    boltzmann,
    white_noise, linear_modes,
    lpt,
    nbody,
    scatter,
    gather
)


In [2]:
##### Spatial parameters
#Number of particles: nc
#Box Size: bs
#Mesh shape: force mesh size/particle grid size

nc = 64
bs = 128 

ptcl_grid_shape = (nc,) * 3
ptcl_spacing = bs/nc
mesh_shape = 1

##### Time parameters
lpt_order = 2
a_start = 0.1
a_stop = 1.0
a_lpt_maxstep = a_start
a_nbody_maxstep = 0.2

#nsteps = 5
#a_nbody_maxstep = (a_stop - a_start)/nsteps

conf = Configuration(ptcl_spacing, ptcl_grid_shape, mesh_shape=mesh_shape,
                     lpt_order=lpt_order, a_start=a_start, a_stop=a_stop, 
                     a_lpt_maxstep=a_lpt_maxstep, a_nbody_maxstep=a_nbody_maxstep)  


print(f'Simulating {conf.ptcl_num} particles with a {conf.mesh_shape} mesh for {conf.a_nbody_num} time steps.')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Simulating 262144 particles with a (64, 64, 64) mesh for 5 time steps.


In [4]:
def simulate(modes, cosmo, conf):
    cosmo = boltzmann(cosmo, conf)
    modes = linear_modes(modes, cosmo, conf)
    ptcl, obsvbl = lpt(modes, cosmo, conf)
    ptcl, obsvbl = nbody(ptcl, obsvbl, cosmo, conf)
    dens = scatter(ptcl, conf)
    return ptcl, obsvbl, dens

In [5]:
n_samp = 10000

#Plank 2018 vals
n_s=0.9665
Omega_b=0.04897
h=0.6766

# varied
A_s_1e9 = np.arange(0.5, 5.51, 0.5) # paper uses sigma_8 but there are some weird bugs when trying to use the 'sigma8' or 'master' branch of pmwd repo 
Omega_m = np.arange(0.05, 0.551, 0.05) 
print('A_s_1e9:', A_s_1e9)
print('Omega_m:', Omega_m)

labels = np.zeros((n_samp, 2)) # contains the randomly selected vals of A_s_1e9 (col 0) and Omega_m (col 1) used to generate the corresponding density field

np.random.seed(0)
for i in tqdm(range(n_samp)):
    rand_idx = int(np.random.rand() * len(A_s_1e9))
    a_s = A_s_1e9[rand_idx]
    rand_idx = int(np.random.rand() * len(Omega_m))
    o_m = Omega_m[rand_idx]
    labels[i, 0] = a_s
    labels[i, 1] = o_m
    
    cosmo = Cosmology(conf, A_s_1e9=a_s, n_s=0.96, Omega_m=o_m, Omega_b=0.05, h=0.7)
    modes = white_noise(i, conf, real=True) # NEED TO CHECK: do I use a different seed for every sample?
    
    _,_, dens = simulate(modes, cosmo, conf)
    dens = dens.sum(axis = 0)
    np.save(f"./data/dens{i:05d}.npy", dens)
np.save(f"./data/labels.npy", labels)

A_s_1e9: [0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5]
Omega_m: [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55]


100%|██████████| 10000/10000 [1:51:13<00:00,  1.50it/s]
